In [1]:
# Adapted from https://github.com/pytorch/examples/tree/master/mnist_hogwild 
from __future__ import print_function
import os

import torch
import torch.optim as optim
import torch.nn.functional as F  # functions are defined, such as relu, softmax, dropout 
import torch.nn as nn  # wherre the layers are defined
import torch.multiprocessing as mp
from torchvision import datasets, transforms


In [2]:
batch_size = 64   # help='input batch size for training (default: 64)'
test_batch_size = 1000   # help='input batch size for testing (default: 1000)'
epochs = 10       # help='number of epochs to train (default: 10)'
lr = 0.01         # help='learning rate (default: 0.01)'
momentum = 0.5    # help='SGD momentum (default: 0.5)'
seed = 1          # help='random seed (default: 1)'
log_interval = 100 # help='how many batches to wait before logging training status'
num_processes = 2 # help='how many training processes to use (default: 2)'
cuda = False      # help='enables CUDA training'

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  ## is in_channel/out_channel the number of nodes? pretty sure yes
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)  
        ## from this, the output chanels are 320, but you need to check the shape to know how to call the next layer
        self.fc1 = nn.Linear(320, 50)
        self.conv2_drop = nn.Dropout2d()
        self.fc1_drop = nn.Dropout()  #alt to F.dropout
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320) ## reshape to rearrange the data, MK does not understand quite how to get this value from their shapes
        x = F.relu(self.fc1(x))
        # x = F.dropout(x, training=self.training)  # possible to use dropout object instead
        x = self.fc1_drop(x)  # p default 0.5, inplate default False
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  #log prob of each of the 10 classes/digits. not compatible with cross-entropy (log_softmax and nll)
    
    # note that if you want to use cross-entropy, use that instead of the nll below (and remove the above usage of log_softmax, instead forward would just return X)

In [6]:
torch.manual_seed(seed)
#use_cuda = cuda and torch.cuda.is_available()
device = torch.device("cpu")  # torch.device("cuda" if use_cuda else "cpu") # 
dataloader_kwargs = {}# {'pin_memory': True} if use_cuda else {}  #maybe buggy, not clear if you need this 


train_loader = torch.utils.data.DataLoader( 
    datasets.MNIST('./data', train=True, download=False,  # set download to True for first run
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, num_workers=1, **dataloader_kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, download=False, # set download to True for first run
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, num_workers=1, **dataloader_kwargs)

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)  # type of optimization algo


In [8]:
model.conv1.weight   ## play with the model's methods and attributes
## use tensor.shape is to check torch.Size 
print(model.conv1.weight.shape)

torch.Size([10, 1, 5, 5])


In [11]:
# Training
for epoch in range(1, epochs + 1):
    print("epoch, ", epoch)
    model.train()
    pid = os.getpid()

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()  # set the gradients to zero to start
        output = model(data.to(device))
        
        # loss is a tensor that is attached to the graph of computation, this is the tape that joseph talked about
        loss = F.nll_loss(output, target.to(device))  #negative log-liklihood loss function (applied to one forward iteration)
        # loss *= 0.1  ## Note: this would actually affect the gradient 
        val = loss.item() * 0.1  ## note that this would store the value, but not change it. Can look at detach for remove. 
        loss.backward() # calculate the gradients to see what to "adjust"
        optimizer.step()  # use the gradients to update the weights
        
        if batch_idx % log_interval == 0:
            print('{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                pid, epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

epoch,  1
20977	Train Epoch: 1 [0/60000 (0%)]	Loss: 0.655943
20977	Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.491729
20977	Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.263191
20977	Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.398660
20977	Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.256739
20977	Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.206893
20977	Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.286191
20977	Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.202419
20977	Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.247192
20977	Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.145374
epoch,  2
20977	Train Epoch: 2 [0/60000 (0%)]	Loss: 0.227303
20977	Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.192070
20977	Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.219509
20977	Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.293381
20977	Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.186252
20977	Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.135143
20977	Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.245980
20977	Train Epoch: 2 [44800/60000 (75%)]

In [12]:
# Test
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():  ## important not to add these gradients to the loss above (will affect model if you do)
    for data, target in test_loader:
        output = model(data.to(device))
        # print(output)
        test_loss += F.nll_loss(output, target.to(device), reduction='sum').item() # sum up batch loss
        # print(test_loss)
        pred = output.max(1)[1] # get the index of the max log-probability  
        correct += pred.eq(target.to(device)).sum().item()  ## this is part of MNIST dataset, ours will be different. 

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))



Test set: Average loss: 0.0457, Accuracy: 9855/10000 (99%)



In [13]:
print(model)
target


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1_drop): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


tensor([5, 8, 6, 5, 8, 2, 1, 6, 1, 5, 4, 0, 1, 1, 4, 5])

In [14]:
print([param.shape for param in model.parameters()])  # this is tricky, spend more time understanding the shapes

[torch.Size([10, 1, 5, 5]), torch.Size([10]), torch.Size([20, 10, 5, 5]), torch.Size([20]), torch.Size([50, 320]), torch.Size([50]), torch.Size([10, 50]), torch.Size([10])]
